# Traffic data analysis

The objective of this notebook is to use data provided by Victor Couture and team to crown "Canada's Worst Commute".

In [1]:
import pandas as pd

In [2]:
# In kilometres.
MIN_TRIP_DISTANCE = 10

# Local hour when rush hour starts.
MORNING_RUSH_HOUR_START = 8
MORNING_RUSH_HOUR_END = 10
EVENING_RUSH_HOUR_START = 17
EVENING_RUSH_HOUR_END = 19

# Minimum number of data points for both rush hour and non-rush hour times.
MIN_DATA_POINTS_NONRUSHHOUR = 10
MIN_DATA_POINTS_RUSHHOUR = 5

# The start and end dates for the 2025 analysis.
# We decided to go back 12 months from Oct. 1, 2025
START_DATE = pd.Timestamp(f"2024-10-01")
END_DATE = pd.Timestamp(f"2025-09-30")

Now we read in the data.

This data has been converted from the original Stata format, which Pandas had some difficulty with. The conversion script is separate, but is fairly simple. The only filtering that was done then was to remove data earlier than Jan. 1, 2024 and to remove columns I know we don't need for this analysis. This was done to keep the file small.

In [3]:
raw = pd.read_parquet('data/victor/alltrips_canada_2024_2025.parquet')

# Create a datetime column for analysis
raw.head(1)

,tripid,mode,citycode,cityname_corrected,time_full_str,traffic_min,notraffic_min,dayofweek,tz,trip_dist,lat_dest,lon_dest,lat_orig,lon_orig
0,101497520,0.0,20370,Calgary,20240111 00:56:12,13.3,13.1,3,America/Edmonton,9.548,51.060974,-113.951439,51.04414,-114.059586


Going to create a copy dataframe here, because I will go back to this raw data at the end for some checks.

In [4]:
df = raw.copy()

We only have a few bits of cleaning to do. Our export method doesn't like é, so we'll change that to e.

There is also a small type in Kitchener-Cambridge-Waterloo that we can fix here.

In [5]:
df["cityname_corrected"] = df["cityname_corrected"].str.replace("é", "e")
df["cityname_corrected"] = df["cityname_corrected"].replace("Kitchener-Cambrigde-Waterloo", "Kitchener-Cambridge-Waterloo")

Now I'm going to create a new column to combine lat/lon for origin and dest. This just makes it easy for me to copy/paste coordinates over to google maps for presentation to the editorial team.

In [6]:
df['origin'] = df['lat_orig'].astype(str) + ',' + df['lon_orig'].astype(str)
df['dest'] = df['lat_dest'].astype(str) + ',' + df['lon_dest'].astype(str)

I will convert the time string into datetime.

Note that the data is already in local timezone, so no conversion is necessary here.

In [7]:
# Create a datetime column for analysis
df['time_dt'] = pd.to_datetime(df['time_full_str'])

df = df.drop(columns=["time_full_str"])

Now we can filter for only the data in the time frames we're interested in.

In [8]:
df = df[(df['time_dt'] <= END_DATE) & (df['time_dt'] >= START_DATE)]

Check data start time and end time.

In [9]:
display(df["time_dt"].min(), df["time_dt"].max())

Timestamp('2024-10-29 05:39:11')

Timestamp('2025-09-29 23:59:46')

What cities is there data for?

In [10]:
df.loc[:, ["cityname_corrected", "tz"]].drop_duplicates()

,cityname_corrected,tz
3970387,Calgary,America/Edmonton
3971133,Edmonton,America/Edmonton
3971843,Halifax,America/Halifax
3972826,Hamilton,America/Toronto
3973695,Kitchener-Cambridge-Waterloo,America/Toronto
3974686,London,America/Toronto
3975805,Montreal,America/Toronto
3977878,Oshawa,America/Toronto
3978757,Ottawa,America/Toronto
3980304,Quebec,America/Toronto


For this table, I’ve excluded routes that are less than 10km long, and only included routes with at least 3 data points collected over the past 6 months in both non-rush hour and rush hour (8-10am, 5-7pm) times. I've excluded everything under 10km, because that doesn't really feel like it qualifies as a commute.

In [11]:
# --- Rush Hour Analysis most recent year
weekdays_df = df[(df['time_dt'].dt.weekday < 5) & (df['trip_dist'] >= MIN_TRIP_DISTANCE) & (df['mode'] == 0)].copy()

hour = weekdays_df['time_dt'].dt.hour

is_rush_hour = ((hour >= MORNING_RUSH_HOUR_START) & (hour < MORNING_RUSH_HOUR_END)) | ((hour >= EVENING_RUSH_HOUR_START) & (hour < EVENING_RUSH_HOUR_END))

weekdays_df['period'] = 'Non-Rush Hour'
weekdays_df.loc[is_rush_hour, 'period'] = 'Rush Hour'

We'll start the analysis here by classifying each data point as a "rush hour" data point, or a "not rush hour" data point.

This is done according to the rush hour cutoffs from above.

Note we are using mean here, not median. This was an editorial decision, and experts have told us that mean and median are both acceptable to use here.

In [12]:
grouping_cols = ['tripid', 'cityname_corrected','period']
agg_times = (weekdays_df
             .groupby(grouping_cols)['traffic_min']
             .agg(['mean', 'count'])
             .reset_index()
             )

agg_times.head(3)

,tripid,cityname_corrected,period,mean,count
0,101497530,Calgary,Non-Rush Hour,24.066667,13
1,101497530,Calgary,Rush Hour,27.936111,6
2,101497531,Calgary,Non-Rush Hour,24.150000,12


Now we reshape a little to collect up all the data points for each tripid.

In [13]:
comparison_table = agg_times.pivot_table(
    index=["tripid", "cityname_corrected"],
    columns='period',
    values=['mean', 'count']
).reset_index()

comparison_table.head(20)

tripid cityname_corrected         count                    mean  \
period                               Non-Rush Hour Rush Hour Non-Rush Hour   
0       101497530            Calgary          13.0       6.0     24.066667   
1       101497531            Calgary          12.0       NaN     24.150000   
2       101497540            Calgary          10.0       6.0     19.196667   
3       101497541            Calgary          13.0       5.0     19.565385   
4       101497551            Calgary          14.0       5.0     13.625000   
5       101497560            Calgary           6.0       1.0     16.913889   
6       101497561            Calgary           9.0       3.0     18.037037   
7       101497570            Calgary          14.0       7.0     24.104762   
8       101497571            Calgary          10.0       3.0     24.615000   
9       101497590            Calgary          11.0       4.0     26.387879   
10      101497591            Calgary          16.0       5.0     25.651042   
11      101497600            Calgary          10.0       4.0     24.968333   
12      101497601            Calgary          13.0       5.0     21.792308   
13      101497620            Calgary          12.0       7.0     27.043056   
14      101497621            Calgary          11.0       2.0     26.839394   
15      101497640            Calgary           4.0       2.0     26.616667   
16      101497641            Calgary          19.0       2.0     23.810526   
17      101497650            Calgary          11.0       3.0     18.022727   
18      101497651            Calgary          10.0       2.0     19.461667   
19      101497660            Calgary           2.0       NaN     13.708333   

                   
period  Rush Hour  
0       27.936111  
1             NaN  
2       21.172222  
3       23.350000  
4       15.130000  
5       17.566667  
6       21.127778  
7       28.421429  
8       29.266667  
9       26.700000  
10      29.396667  
11      33.183333  
12      27.456667  
13      34.133333  
14      28.666667  
15      30.366667  
16      29.691667  
17      20.050000  
18      20.233333  
19            NaN

Now we just flatten the columns so they're a bit cleaner.

In [14]:
# Flatten multiindex column names
comparison_table.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in comparison_table.columns.values]

comparison_table.head(3)

,tripid_,cityname_corrected_,count_Non-Rush Hour,count_Rush Hour,mean_Non-Rush Hour,mean_Rush Hour
0,101497530,Calgary,13.0,6.0,24.066667,27.936111
1,101497531,Calgary,12.0,NaN,24.150000,NaN
2,101497540,Calgary,10.0,6.0,19.196667,21.172222


And rename them because they're a bit messy.

In [15]:
comparison_table.rename(columns={
    'mean_Non-Rush Hour': 'mean_non_rush_hour',
    'mean_Rush Hour': 'mean_rush_hour',
    'count_Non-Rush Hour': 'count_non_rush_hour',
    'count_Rush Hour': 'count_rush_hour',
    'tripid_': 'tripid',
    'mode_': 'mode',
    'cityname_corrected_': 'cityname_corrected',
    'trip_dist_': 'trip_dist',
}, inplace=True)

Now I want to know what the average travel time is for each trip with no traffic.

In [16]:
# Compute a single overall mean for notraffic_min (include all periods)
notraffic_overall = (weekdays_df
                     .groupby(['tripid', 'cityname_corrected'])['notraffic_min']
                     .mean()
                     .reset_index()
                     .rename(columns={'notraffic_min': 'mean_notraffic'})
                     )

notraffic_overall.head(3)

,tripid,cityname_corrected,mean_notraffic
0,101497530,Calgary,22.245614
1,101497531,Calgary,21.687500
2,101497540,Calgary,17.211458


And because it's of interest to the story team, I'd like to include a column that shows the 90th percentile time for each trip.

In [17]:
# Compute 90th percentile travel time during Rush Hour per trip (p90)
rush_df = weekdays_df[weekdays_df['period'] == 'Rush Hour']
p90_rush = (rush_df
            .groupby(['tripid', 'cityname_corrected'])['traffic_min']
            .quantile(0.9)
            .reset_index()
            .rename(columns={'traffic_min': 'p90_rush_hour'})
            )

p90_rush.head(3)

,tripid,cityname_corrected,p90_rush_hour
0,101497530,Calgary,32.450000
1,101497540,Calgary,24.583333
2,101497541,Calgary,26.476667


And I'd also like to know roughly how long each trip is. Note that each trip appears to be a slightly different distance, even within the same route ideas. This is because Google Maps may take slightly different routes each time.

In [18]:
# Compute a single overall mean for notraffic_min (include all periods)
tripdist = (weekdays_df
                     .groupby(['tripid', 'cityname_corrected'])['trip_dist']
                     .mean()
                     .reset_index()
                     .rename(columns={'trip_dist': 'mean_trip_dist'})
                     )

tripdist.head(3)

,tripid,cityname_corrected,mean_trip_dist
0,101497530,Calgary,18.731105
1,101497531,Calgary,20.683417
2,101497540,Calgary,12.493187


Now we merge the overall notraffic mean, p90, and tripdist tables into the comparison table

In [19]:
comparison_table = comparison_table.merge(notraffic_overall, on=['tripid', 'cityname_corrected'], how='left')
comparison_table = comparison_table.merge(p90_rush, on=['tripid', 'cityname_corrected'], how='left')
comparison_table = comparison_table.merge(tripdist, on=['tripid', 'cityname_corrected'], how='left')

comparison_table.head(3)

,tripid,cityname_corrected,count_non_rush_hour,count_rush_hour,mean_non_rush_hour,mean_rush_hour,mean_notraffic,p90_rush_hour,mean_trip_dist
0,101497530,Calgary,13.0,6.0,24.066667,27.936111,22.245614,32.450000,18.731105
1,101497531,Calgary,12.0,NaN,24.150000,NaN,21.687500,NaN,20.683417
2,101497540,Calgary,10.0,6.0,19.196667,21.172222,17.211458,24.583333,12.493187


Origin and destination should be the exact same for each trip id. But in case they differ slightly, we'll just grab the first of each pulled for each route id and add to our table.

In [20]:
orig_dest = weekdays_df.groupby(['tripid', 'cityname_corrected']).agg({'origin': 'first', 'dest': 'first'}).reset_index()
comparison_table = comparison_table.merge(orig_dest, on=['tripid', 'cityname_corrected'], how='left')

comparison_table.head(3)

,tripid,cityname_corrected,count_non_rush_hour,count_rush_hour,mean_non_rush_hour,mean_rush_hour,mean_notraffic,p90_rush_hour,mean_trip_dist,origin,dest
0,101497530,Calgary,13.0,6.0,24.066667,27.936111,22.245614,32.450000,18.731105,"51.04198,-114.051346","51.12566,-114.22575"
1,101497531,Calgary,12.0,NaN,24.150000,NaN,21.687500,NaN,20.683417,"51.12566,-114.22575","51.04198,-114.051346"
2,101497540,Calgary,10.0,6.0,19.196667,21.172222,17.211458,24.583333,12.493187,"51.044357,-114.061646","50.949066,-114.085335"


Now we'll calculate some columns. We start with the difference in travel time between freeflow (no traffic) and rush hour.

In [21]:
comparison_table['rush_vs_freeflow_diff'] = comparison_table['mean_rush_hour'] - comparison_table['mean_notraffic']

comparison_table.head(3)

,tripid,cityname_corrected,count_non_rush_hour,count_rush_hour,mean_non_rush_hour,mean_rush_hour,mean_notraffic,p90_rush_hour,mean_trip_dist,origin,dest,rush_vs_freeflow_diff
0,101497530,Calgary,13.0,6.0,24.066667,27.936111,22.245614,32.450000,18.731105,"51.04198,-114.051346","51.12566,-114.22575",5.690497
1,101497531,Calgary,12.0,NaN,24.150000,NaN,21.687500,NaN,20.683417,"51.12566,-114.22575","51.04198,-114.051346",NaN
2,101497540,Calgary,10.0,6.0,19.196667,21.172222,17.211458,24.583333,12.493187,"51.044357,-114.061646","50.949066,-114.085335",3.960764


Now rush hour time versus freeflow PERCENT.

This is the percent increase in travel time, on average, during rush hour versus free flow.

In [22]:
comparison_table['rush_vs_freeflow_pct'] = comparison_table.apply(lambda r: (r['rush_vs_freeflow_diff'] / r['mean_notraffic'] * 100) if r['mean_notraffic'] else 0, axis=1)

comparison_table.head(3)

,tripid,cityname_corrected,count_non_rush_hour,count_rush_hour,mean_non_rush_hour,mean_rush_hour,mean_notraffic,p90_rush_hour,mean_trip_dist,origin,dest,rush_vs_freeflow_diff,rush_vs_freeflow_pct
0,101497530,Calgary,13.0,6.0,24.066667,27.936111,22.245614,32.450000,18.731105,"51.04198,-114.051346","51.12566,-114.22575",5.690497,25.580310
1,101497531,Calgary,12.0,NaN,24.150000,NaN,21.687500,NaN,20.683417,"51.12566,-114.22575","51.04198,-114.051346",NaN,NaN
2,101497540,Calgary,10.0,6.0,19.196667,21.172222,17.211458,24.583333,12.493187,"51.044357,-114.061646","50.949066,-114.085335",3.960764,23.012367


Now finall we compare the 90th percentile travel time to freeflow by subtraction.

In [23]:
comparison_table['p90_vs_freeflow'] = comparison_table['p90_rush_hour'] - comparison_table['mean_notraffic']

comparison_table.head(3)

,tripid,cityname_corrected,count_non_rush_hour,count_rush_hour,mean_non_rush_hour,mean_rush_hour,mean_notraffic,p90_rush_hour,mean_trip_dist,origin,dest,rush_vs_freeflow_diff,rush_vs_freeflow_pct,p90_vs_freeflow
0,101497530,Calgary,13.0,6.0,24.066667,27.936111,22.245614,32.450000,18.731105,"51.04198,-114.051346","51.12566,-114.22575",5.690497,25.580310,10.204386
1,101497531,Calgary,12.0,NaN,24.150000,NaN,21.687500,NaN,20.683417,"51.12566,-114.22575","51.04198,-114.051346",NaN,NaN,NaN
2,101497540,Calgary,10.0,6.0,19.196667,21.172222,17.211458,24.583333,12.493187,"51.044357,-114.061646","50.949066,-114.085335",3.960764,23.012367,7.371875


If we have no data points in rush hour, let's just drop that trip.

In [24]:
comparison_table = comparison_table[~comparison_table['count_rush_hour'].isna()]

In [25]:
# Convert count columns to integers where present
for col in ['count_rush_hour']:
    if col in comparison_table.columns:
        comparison_table[col] = comparison_table[col].astype(int)

Now we drop anything without enough data points, defined our assumptions constants above.

In [26]:
print(f"Before dropping min data points: {len(comparison_table)}.")
comparison_table = comparison_table[(comparison_table['count_rush_hour'] >= MIN_DATA_POINTS_RUSHHOUR)]
print(f"After dropping min data points: {len(comparison_table)}.")

Before dropping min data points: 58119.
After dropping min data points: 17235.


Now make sure count columns are integers.

In [27]:
# These columns should be floats of 1 decimal.
comparison_table['mean_trip_dist'] = comparison_table['mean_trip_dist'].round(1)

# These columns should be floats of 2 decimals.
num_cols = ['mean_non_rush_hour', 'mean_rush_hour', 'mean_notraffic', 'rush_vs_freeflow_diff', 'rush_vs_freeflow_pct', 'p90_rush_hour']

for c in num_cols:
    if c in comparison_table.columns:
        comparison_table[c] = comparison_table[c].round(2)

In [28]:
# Final sorting and filtering
final_table = comparison_table.sort_values('rush_vs_freeflow_pct', ascending=False)

final_table.head(3)

,tripid,cityname_corrected,count_non_rush_hour,count_rush_hour,mean_non_rush_hour,mean_rush_hour,mean_notraffic,p90_rush_hour,mean_trip_dist,origin,dest,rush_vs_freeflow_diff,rush_vs_freeflow_pct,p90_vs_freeflow
30984,201307011,Montreal,15.0,6,30.94,41.73,17.05,53.60,12.5,"45.610847,-73.5166","45.57193,-73.45377",24.67,144.67,36.545238
32050,201317100,Montreal,13.0,5,33.74,45.01,18.45,51.99,13.8,"45.607246,-73.51454","45.5652,-73.45309",26.56,143.93,33.542407
14171,101623691,Toronto,10.0,5,23.72,46.75,19.30,54.39,18.1,"43.59524,-79.54329","43.651657,-79.3833",27.45,142.24,35.091111


I'm going to now "name" the top routes currently in this table.

In [29]:
names = [
    {
    "tripid": "101622821",
    "name": "Downtown to Oakville (1018 Pinewood Rd.)"
    },
    {
    "tripid": "101615481",
    "name": "Downtown to Oakville (2001 Speers Rd.)"
    },
    {
    "tripid": "101627951",
    "name": "Downtown to Burlington"
    },
    {
    "tripid": "101612671",
    "name": "Downtown to Oakville (1393 Harmsworth Square)"
    },
    {
    "tripid": "101622220",
    "name": "Downtown to Brampton"
    },
    {
    "tripid": "101621581",
    "name": "Downtown to Oakville (155 Waterstone Court)"
    },
    {
    "tripid": "101608211",
    "name": "Downtown to Oakville (1136 Queens Avenue)"
    },
    {
    "tripid": "101627971",
    "name": "Downtown to Burlington (2094 Deer Run Avenue)"
    },
    {
    "tripid": "101619411",
    "name": "Downtown to Oakville (1381 Creekwood Trail)"
    },
    {
    "tripid": "101609371",
    "name": "Downtown to Burlington (489 Trillium Drive)"
    },
 ]

Now we stitch this onto the table.

In [30]:
name_df = pd.DataFrame(names).set_index("tripid")

final_table_with_names = final_table.set_index("tripid",).join(name_df, how="left")

final_table_with_names.head(len(name_df))

,cityname_corrected,count_non_rush_hour,count_rush_hour,mean_non_rush_hour,mean_rush_hour,mean_notraffic,p90_rush_hour,mean_trip_dist,origin,dest,rush_vs_freeflow_diff,rush_vs_freeflow_pct,p90_vs_freeflow,name
tripid,,,,,,,,,,,,,,
201307011,Montreal,15.0,6,30.94,41.73,17.05,53.60,12.5,"45.610847,-73.5166","45.57193,-73.45377",24.67,144.67,36.545238,NaN
201317100,Montreal,13.0,5,33.74,45.01,18.45,51.99,13.8,"45.607246,-73.51454","45.5652,-73.45309",26.56,143.93,33.542407,NaN
101623691,Toronto,10.0,5,23.72,46.75,19.30,54.39,18.1,"43.59524,-79.54329","43.651657,-79.3833",27.45,142.24,35.091111,NaN
14176741,Toronto,21.0,5,28.17,47.71,20.06,62.91,18.8,"43.594994,-79.54638","43.653397,-79.38021",27.65,137.89,42.854231,NaN
14178751,Toronto,16.0,8,28.07,45.81,19.50,51.32,20.8,"43.653893,-79.56732","43.649918,-79.38193",26.31,134.92,31.825694,NaN
303925231,Montreal,14.0,5,14.55,23.25,9.98,26.88,10.6,"45.483418,-73.721565","45.472343,-73.61273",13.27,132.91,16.894211,NaN
303934731,Montreal,11.0,5,20.76,28.47,12.30,38.10,10.9,"45.52769,-73.65153","45.538033,-73.638824",16.17,131.51,25.804375,NaN
303944041,Montreal,12.0,5,34.67,45.66,19.77,59.55,26.2,"45.576736,-73.76414","45.44489,-73.64329",25.89,130.98,39.785686,NaN
14178800,Toronto,5.0,5,20.88,31.95,13.86,45.53,16.7,"43.653397,-79.37472","43.612644,-79.55153",18.09,130.55,31.668333,NaN


Add a column for rank, to make it easier to understand for the story team.

In [31]:
final_table_with_names["rank"] = range(1,len(final_table_with_names)+1)

Now let's take a look at the final table, with rearranged column order in way that makes more sense.

In [32]:
display_cols = ["rank", 'tripid', "name", 'cityname_corrected', 'origin', 'dest', 'mean_trip_dist', 'mean_notraffic', 'mean_rush_hour', 'rush_vs_freeflow_diff', 'rush_vs_freeflow_pct', 'p90_rush_hour', "p90_vs_freeflow"]

In [36]:
final_table_with_names.reset_index().set_index("rank")[[col for col in display_cols if col != "rank"]].head(20)

,tripid,name,cityname_corrected,origin,dest,mean_trip_dist,mean_notraffic,mean_rush_hour,rush_vs_freeflow_diff,rush_vs_freeflow_pct,p90_rush_hour,p90_vs_freeflow
rank,,,,,,,,,,,,
1,201307011,NaN,Montreal,"45.610847,-73.5166","45.57193,-73.45377",12.5,17.05,41.73,24.67,144.67,53.60,36.545238
2,201317100,NaN,Montreal,"45.607246,-73.51454","45.5652,-73.45309",13.8,18.45,45.01,26.56,143.93,51.99,33.542407
3,101623691,NaN,Toronto,"43.59524,-79.54329","43.651657,-79.3833",18.1,19.30,46.75,27.45,142.24,54.39,35.091111
4,14176741,NaN,Toronto,"43.594994,-79.54638","43.653397,-79.38021",18.8,20.06,47.71,27.65,137.89,62.91,42.854231
5,14178751,NaN,Toronto,"43.653893,-79.56732","43.649918,-79.38193",20.8,19.50,45.81,26.31,134.92,51.32,31.825694
6,303925231,NaN,Montreal,"45.483418,-73.721565","45.472343,-73.61273",10.6,9.98,23.25,13.27,132.91,26.88,16.894211
7,303934731,NaN,Montreal,"45.52769,-73.65153","45.538033,-73.638824",10.9,12.30,28.47,16.17,131.51,38.10,25.804375
8,303944041,NaN,Montreal,"45.576736,-73.76414","45.44489,-73.64329",26.2,19.77,45.66,25.89,130.98,59.55,39.785686
9,14178800,NaN,Toronto,"43.653397,-79.37472","43.612644,-79.55153",16.7,13.86,31.95,18.09,130.55,45.53,31.668333


In [34]:
final_table_with_names.reset_index().set_index("rank")[[col for col in display_cols if col != "rank"]].to_csv("results-victor.csv")

## Mapping results

In [35]:
# # Save top-3 route maps per city and fetch fresh geometry from Google Directions for each origin/destination
# import os
# import pathlib
# import re
# import time
# import requests
# import polyline
# import matplotlib.pyplot as plt
# import geopandas as gpd
# from shapely.geometry import LineString

# out_dir = pathlib.Path('plots_by_route')
# out_dir.mkdir(parents=True, exist_ok=True)

# # Read API key (file fallback then env)
# API_KEY = None
# key_path = os.path.join(os.getcwd(), 'google_maps_api_key.txt')
# if os.path.exists(key_path):
#     with open(key_path, 'r', encoding='utf-8') as fh:
#         API_KEY = fh.read().strip()
# if not API_KEY:
#     API_KEY = os.getenv('GOOGLE_API_KEY')
# if not API_KEY:
#     raise RuntimeError('Google Maps API key not found. Add google_maps_api_key.txt or set GOOGLE_API_KEY')

# def safe(s):
#     if s is None:
#         return 'unknown'
#     s = str(s)
#     s = re.sub(r'[^A-Za-z0-9_\-]', '_', s)
#     return s[:180]

# if 'final_table' not in globals():
#     raise RuntimeError('`final_table` not found. Run the Rush Hour Analysis cell first.')

# ft = final_table.reset_index()
# if 'rank' not in ft.columns:
#     ft.insert(0, 'rank', range(1, len(ft) + 1))

# # cache to avoid duplicate API calls for identical origin/dest within this run
# route_cache = {}

# def fetch_route_geometry(origin_str, dest_str, max_retries=3):
#     key = f'{origin_str}|{dest_str}'
#     if key in route_cache:
#         return route_cache[key]
#     url = 'https://maps.googleapis.com/maps/api/directions/json'
#     params = {
#         'origin': origin_str,
#         'destination': dest_str,
#         'alternatives': 'false',
#         'mode': 'driving',
#         'key': API_KEY
#     }
#     backoff = 1.0
#     for attempt in range(1, max_retries + 1):
#         try:
#             resp = requests.get(url, params=params, timeout=15)
#             resp.raise_for_status()
#             j = resp.json()
#         except Exception as e:
#             if attempt == max_retries:
#                 raise
#             time.sleep(backoff)
#             backoff *= 2
#             continue
#         status = j.get('status')
#         if status != 'OK' or not j.get('routes'):
#             # For non-retryable API errors, return None
#             return None
#         ov = j['routes'][0].get('overview_polyline', {}).get('points')
#         if not ov:
#             return None
#         coords_latlon = polyline.decode(ov)  # list of (lat, lon)
#         coords = [(lon, lat) for lat, lon in coords_latlon]
#         ls = LineString(coords)
#         # create polygon buffer of 100 m by projecting to web mercator
#         single_line_gdf = gpd.GeoDataFrame([{'tripid': None, 'origin': origin_str, 'dest': dest_str, 'linestring': ls}], geometry='linestring', crs='EPSG:4326')
#         web = single_line_gdf.to_crs(epsg=3857)
#         web['polygon_100m'] = web.geometry.buffer(100)
#         poly = gpd.GeoDataFrame(web.copy(), geometry=web['polygon_100m'], crs=web.crs).to_crs(epsg=4326)
#         route_cache[key] = (ls, poly)
#         return ls, poly
#     return None

# for city in ft['cityname_corrected'].unique():
#     city_rows = ft[ft['cityname_corrected'] == city].sort_values('rush_vs_freeflow_diff', ascending=False).head(3)
#     if city_rows.empty:
#         continue
#     for _, crow in city_rows.iterrows():
#         rank = int(crow.get('rank', -1))
#         tripid = crow.get('tripid')
#         origin_val = crow.get('origin')
#         dest_val = crow.get('dest')
#         if not origin_val or not dest_val:
#             print(f'Missing origin/dest for trip {tripid} (rank {rank}) in {city}; skipping')
#             continue
#         # origin_val/dest_val should be 'lat,lon' strings; ensure trimmed
#         origin_str = str(origin_val).strip()
#         dest_str = str(dest_val).strip()
#         try:
#             res = fetch_route_geometry(origin_str, dest_str)
#         except Exception as e:
#             print(f'Error fetching route for {tripid} in {city}: {e}; skipping')
#             continue
#         if not res:
#             print(f'No route geometry returned for {tripid} in {city}; skipping')
#             continue
#         ls, poly = res
#         fname = out_dir / f"{rank:03d}_{safe(tripid)}_{safe(city)}.png"
#         fig, ax = plt.subplots(1, 1, figsize=(8, 6))
#         try:
#             if poly is not None and not poly.empty:
#                 poly.to_crs(epsg=3857).plot(ax=ax, color='orange', alpha=0.4, edgecolor='darkorange')
#             single_line = gpd.GeoDataFrame([{'linestring': ls}], geometry='linestring', crs='EPSG:4326')
#             single_line.to_crs(epsg=3857).plot(ax=ax, color='red', linewidth=3)
#             minx, miny, maxx, maxy = single_line.to_crs(epsg=3857).total_bounds
#             buf = max((maxx - minx), (maxy - miny)) * 0.25 if maxx > minx and maxy > miny else 200
#             ax.set_xlim(minx - buf, maxx + buf)
#             ax.set_ylim(miny - buf, maxy + buf)
#             try:
#                 import contextily as ctx
#                 ctx.add_basemap(ax)
#             except Exception:
#                 pass
#             ax.set_axis_off()
#             ax.set_title(f'Rank {rank} — Trip {tripid} — {city}')
#             fig.savefig(fname, dpi=150, bbox_inches='tight')
#             plt.close(fig)
#             print(f'Saved {fname}')
#         except Exception as e:
#             print(f'Failed to save route {tripid} (rank {rank}) for city {city}: {e}')
